In [1]:
from keras.models import load_model

# returns a compiled model
# identical to the previous one
model = load_model('emotion_model_sequential.h5')

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load pre-trained face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the saved emotion detection model
model = tf.keras.models.load_model('emotion_model_sequential.h5')

# Emotion labels corresponding to model's output classes
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Model input size (expected by your model)
input_size = (224, 224)  # Adjust to match the size used during training

# Initialize camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]  # Crop the face region
        
        # Resize to match the input size of the model
        face = cv2.resize(face, input_size)  # Resize to 224x224 (or model's expected size)
        
        # Normalize the pixel values to [0, 1]
        face = face / 255.0
        
        # Add channel and batch dimensions for grayscale (single channel)
        face = np.expand_dims(face, axis=-1)  # Add channel dimension (for grayscale, 1 channel)
        face = np.expand_dims(face, axis=0)  # Add batch dimension (for single image)
        
        # Predict emotion
        predictions = model.predict(face)
        emotion = emotion_labels[np.argmax(predictions[0])]

        # Draw bounding box and label on the frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Display the frame with predicted emotion
    cv2.imshow('Classification Emotion Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
